<a href="https://colab.research.google.com/github/jnjorstad/Air-Quality-and-Lung-Cancer/blob/main/Apparel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# find the latest version of spark 3.0
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION'] = spark_version

# install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# start a sparksession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 6s (42.8 kB/s)
Reading package li

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-08-12 02:57:33--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.2’

postgresql-42.2.9.j 100%[===================>] 892.61K   852KB/s    in 1.0s    

2022-08-12 02:57:34 (852 KB/s) - ‘postgresql-42.2.9.jar.2’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloutETL").config("spark.drive.extraClassPath","/content/postgresql-42.2.9.jar.1").getOrCreate()

In [4]:
# read in data from S3 buckets

from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

apparel_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Apparel_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat = "mm/dd/yy")

# show DataFrame
apparel_df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [5]:
# drop duplicates and incomplete rows
print(apparel_df.count())
apparel_df = apparel_df.dropna()
print(apparel_df.count())
apparel_df = apparel_df.dropDuplicates()
print(apparel_df.count())

5906333
5905269
5905269


In [6]:
# examine the schema
apparel_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
# load in a sql function to use columns
from pyspark.sql.functions import col

In [8]:
# create DataFrame to match review_id_table
apparel_review_df = apparel_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
apparel_review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1000LG8H70CNQ|     696306|B00O3A0RHY|     246430568| 2015-01-25|
|R100BCEUAIHZQC|   11807812|B00CASTERA|     600575877| 2014-06-03|
|R100EOB2GC5UYQ|   15017601|B002MLE36W|     988639002| 2014-10-26|
|R100MYSZIXWX1D|   44291413|B007HSEDVK|        174345| 2012-10-31|
|R100UMGHMFVXKC|   24539177|B00F52CUCE|     162326772| 2014-03-18|
|R101CJDLDDM19P|   10041263|B00NIX31K2|     559380483| 2014-12-16|
|R101OE6ZEL1A52|   20521864|B00CB1S40E|     248821115| 2014-02-08|
| R101PL938YJV0|   44457530|B00AF7D7VC|     656221590| 2013-11-14|
|R101QMSD2TQTAP|   14103889|B00N9KGZWA|     906968721| 2015-04-30|
|R101V43VD0FQ5Y|   23446176|B007WPF25E|     619910868| 2013-07-01|
|R102HAEXYCF1YM|   20169548|B00DECZ2KE|     913386738| 2014-02-09|
|R102KCQDXFRN3E|   18522476|B000092MPB|     673568662| 2006-06

In [9]:
#create DataFrame to match products table
apparel_products_df = apparel_df.select(["product_id", "product_title"])
apparel_products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00O3A0RHY|Giant Panda Face ...|
|B00CASTERA|  Rothco Jungle Hats|
|B002MLE36W|Leg Avenue Women'...|
|B007HSEDVK|LAST KISS Ombré W...|
|B00F52CUCE|Magiftbox Men's S...|
|B00NIX31K2|ROEWELL® 4 Sets o...|
|B00CB1S40E|Valco Baby Snap4 ...|
|B00AF7D7VC|Louis Raphael ROS...|
|B00N9KGZWA|Women`s Push Up T...|
|B007WPF25E|Kangol Men's Josh...|
|B00DECZ2KE|London Times Wome...|
|B000092MPB|      Zip-front suit|
|B00CFET4Y2|Kanu Surf Men's C...|
|B00O3MJRRS|Modern Minute - N...|
|B003N63ET2|Champion Women's ...|
|B00IQ72SVS|Lg Dk. Green Semi...|
|B004BLAC0M|World's Softest M...|
|B00EY9QS9U|Zeagoo Women's Ch...|
|B00CBKJN64|The Beatles - Gre...|
|B00MUGCZA0|Inktastic - Red F...|
+----------+--------------------+
only showing top 20 rows



In [10]:
#create DataFrame to match customer table
apparel_customers_df = apparel_df.groupBy('customer_id').agg({"customer_id": "count"})
apparel_customers_df = apparel_customers_df.withColumnRenamed("count(customer_id)", "customer_count")
#furniture_customers_df = furniture_df.select(["customer_id", "customer_count"])
apparel_customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12955905|             5|
|    3909801|             1|
|   17063052|             1|
|   13245107|             1|
|   15149043|             5|
|    5137716|             1|
|   42673458|            23|
|   15476378|             1|
|   11615457|             1|
|   52209445|             2|
|   47277612|            18|
|     481101|             1|
|    3207957|             2|
|   20874267|             3|
|   21580335|             2|
|    4843304|            26|
|   36715887|             2|
|    4416285|            18|
|   31663336|             7|
|   15296593|             2|
+-----------+--------------+
only showing top 20 rows



In [19]:
apparel_review_df

DataFrame[review_id: string, customer_id: int, product_id: string, product_parent: int, review_date: string]

In [14]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://endpoint:5432/big_data_hw_2"
config = {"user":"postgres", 
          "password": "password", 
          "driver":"org.postgresql.Driver"}

In [16]:
 # Write DataFrame to review_id_table in RDS

apparel_review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

Py4JJavaError: ignored

In [17]:
 # Write dataframe to products table in RDS

apparel_products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

Py4JJavaError: ignored

In [18]:
 # Write dataframe to customers table in RDS

apparel_customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

Py4JJavaError: ignored